In [ ]:
"""TrabalhoIA2.ipynb

Automatically generated by Colab.

Original file is located at
    https://colab.research.google.com/drive/1ICKWHmssmG1nmBGYllxTiMEY5lHPRxI9
"""

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00222/bank.zip"
!wget -q $url       # baixa o ZIP
!unzip -q bank.zip  # extrai bank-full.csv e outros

In [ ]:
df = pd.read_csv("bank-full.csv", sep=";")
df.head()

In [ ]:
import tensorflow_datasets as tfds

In [ ]:
(ds_train, ds_val, ds_test), info = tfds.load(
    "tf_flowers",
    split=["train[:80%]", "train[80%:90%]", "train[90%:]"],
    as_supervised=True,
    with_info=True,
)
print("Classes:", info.features["label"].names)

In [ ]:
import zipfile
import os
import pandas as pd

Etapa 1: Upload manual do arquivo ZIP no Colab
from google.colab import files
uploaded = files.upload()

In [ ]:
# Etapa 2: Extrair o ZIP externo
outer_zip_path = '/content/bank+marketing.zip'
outer_extract_path = '/content/bank_outer'
with zipfile.ZipFile(outer_zip_path, 'r') as outer_zip:
    outer_zip.extractall(outer_extract_path)

In [ ]:
# Etapa 3: Extrair o ZIP interno (bank.zip)
inner_zip_path = os.path.join(outer_extract_path, 'bank.zip')
inner_extract_path = '/content/bank_data'
with zipfile.ZipFile(inner_zip_path, 'r') as inner_zip:
    inner_zip.extractall(inner_extract_path)

In [ ]:
# Etapa 4: Listar e carregar o CSV principal
print("Arquivos disponíveis:", os.listdir(inner_extract_path))

In [ ]:
df = pd.read_csv(os.path.join(inner_extract_path, 'bank.csv'), sep=';')
df.head()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
# 1. Converter variáveis binárias para 0/1
df_bin = df.copy()
binary_map = {'yes': 1, 'no': 0}
for col in ['default', 'housing', 'loan', 'y']:
    df_bin[col] = df_bin[col].map(binary_map)

In [ ]:
# 2. One-hot encoding nas variáveis categóricas
categorical_cols = ['job', 'marital', 'education', 'contact', 'month', 'poutcome']
df_encoded = pd.get_dummies(df_bin, columns=categorical_cols, drop_first=True)

In [ ]:
# 3. Separar X (features) e y (rótulo)
X = df_encoded.drop('y', axis=1)
y = df_encoded['y']

In [ ]:
# 4. Padronizar colunas numéricas
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:
# 5. Dividir em treino e teste (80/20)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [ ]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, accuracy_score

In [ ]:
# Lista de configurações de hiperparâmetros
params_list = [
    {"criterion": "gini", "max_depth": None},
    {"criterion": "entropy", "max_depth": 5},
    {"criterion": "gini", "max_depth": 10, "min_samples_split": 10}
]

In [ ]:
# Lista para guardar resultados
results_dt = []

In [ ]:
for i, params in enumerate(params_list, 1):
    clf = DecisionTreeClassifier(random_state=42, **params)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    report = classification_report(y_test, y_pred, output_dict=True, zero_division=0)
    acc = accuracy_score(y_test, y_pred)

    results_dt.append({
        "Configuração": f"Configuração {i}",
        "Hiperparâmetros": params,
        "Acurácia": acc,
        "Precisão (classe 1)": report['1']['precision'],
        "Recall (classe 1)": report['1']['recall'],
        "F1-score (classe 1)": report['1']['f1-score']
    })

In [ ]:
# Mostrar resultados
import pandas as pd
pd.DataFrame(results_dt)

In [ ]:
"""# Task
Carrega o conjunto de resenhas de livros em português para classificar sentimentos.

Here is all the data you need:
"archive/books_reviews.csv"

## Data loading

### Subtask:
Load the dataset "archive/books_reviews.csv" into a pandas DataFrame.

**Reasoning**:
Carregar a base de resenhas já combinada em um único CSV local e exibir informações básicas.
"""

In [ ]:
import pandas as pd

In [ ]:
try:
    df = pd.read_csv('archive/books_reviews.csv')
except UnicodeDecodeError:
    try:
        df = pd.read_csv('archive/books_reviews.csv', encoding='latin-1')
    except UnicodeDecodeError:
        try:
            df = pd.read_csv('archive/books_reviews.csv', encoding='utf-8')
        except Exception as e:
            print(f"Error loading the file: {e}")
            df = None

In [ ]:
if df is not None:
    display(df.head())
    print(df.shape)
    df.info()

In [ ]:
"""## Data exploration

### Subtask:
Explorar rapidamente a base de resenhas, verificando quantidade de exemplos por classe e tamanho médio dos textos.

**Reasoning**:
Como o conjunto possui apenas duas colunas (texto e rótulo), a análise pode ser simplificada para foco na distribuição das classes e nas estatísticas básicas das resenhas.
"""

In [ ]:
print("Shape of the DataFrame:", df.shape)
print(df['label'].value_counts())

In [ ]:
# calcular o comprimento médio das resenhas
df['text_length'] = df['review_text'].astype(str).apply(len)
print("Tamanho médio do texto:", df['text_length'].mean())

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
plt.figure(figsize=(6,4))
sns.countplot(x='label', data=df)
plt.title('Distribuição das classes')
plt.show()

In [ ]:
plt.figure(figsize=(6,4))
sns.histplot(df['text_length'], kde=True)
plt.title('Distribuição do tamanho das resenhas')
plt.show()

In [ ]:
"""# Task
eu tenho uma base aqui que eu preciso subir para cá e que você consiga fazer a leitura desses dados para fazer um pré-processamento dos dados para depois aplicar os métodos como de árvore de decisão e quem saber uma rede neural, como eu subo?

Here is all the data you need:
"B2W-Reviews01.csv"

## Data loading

### Subtask:
Load the dataset "B2W-Reviews01.csv" into a pandas DataFrame.

**Reasoning**:
Load the dataset "B2W-Reviews01.csv" into a pandas DataFrame and display basic information about it.
"""

In [ ]:
import pandas as pd

In [ ]:
try:
    df = pd.read_csv('B2W-Reviews01.csv')
except UnicodeDecodeError:
    try:
        df = pd.read_csv('B2W-Reviews01.csv', encoding='latin-1')
    except UnicodeDecodeError:
        try:
            df = pd.read_csv('B2W-Reviews01.csv', encoding='utf-8')
        except Exception as e:
            print(f"Error loading the file: {e}")
            df = None

In [ ]:
if df is not None:
    display(df.head())
    print(df.shape)
    df.info()

In [ ]:
"""## Data exploration

### Subtask:
Explore the dataset to understand its structure, identify data types of each column, check for missing values, and examine the distribution of key variables. Determine the shape of the data and look for potential issues like inconsistencies or irrelevant columns.  Pay special attention to the 'review_comment_title' and 'review_comment_message' columns, as these might contain valuable textual information.

**Reasoning**:
Explore the dataset, identify data types, check for missing values, examine the distribution of key variables, and look for potential issues.  This involves using .shape, .info(), .describe(), examining unique values for categorical columns, calculating text lengths, and checking for duplicates.
"""

In [ ]:
# Check the shape of the DataFrame
print("Shape of the DataFrame:", df.shape)

In [ ]:
# Get a summary of the DataFrame
df.info()

In [ ]:
# Descriptive statistics for numerical columns
print("\nDescriptive Statistics:")
display(df.describe())

In [ ]:
# Examine unique values for categorical columns
categorical_cols = df.select_dtypes(include=['object']).columns
for col in categorical_cols:
    print(f"\nUnique values and frequencies for {col}:")
    print(df[col].value_counts())

In [ ]:
# Calculate the average length of text in 'review_comment_title' and 'review_comment_message'
# Note: These columns are 'review_title' and 'review_text' in the provided data.
if 'review_title' in df.columns:
    df['review_title_length'] = df['review_title'].astype(str).apply(len)
    print(f"\nAverage length of review_title: {df['review_title_length'].mean()}")
if 'review_text' in df.columns:
    df['review_text_length'] = df['review_text'].astype(str).apply(len)
    print(f"\nAverage length of review_text: {df['review_text_length'].mean()}")

In [ ]:
# Check for duplicated rows
duplicated_rows = df.duplicated().sum()
print(f"\nNumber of duplicated rows: {duplicated_rows}")

In [ ]:
"""**Reasoning**:
Visualize the distribution of key variables using histograms, box plots, and count plots. Pay special attention to the `overall_rating` column. Check for potential outliers or unusual patterns in numerical and categorical features. Also, visualize the correlation matrix using a heatmap.


"""

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# --- carregar ---
df = pd.read_csv("archive/books_reviews.csv")           # ajuste o caminho se necessário

In [ ]:
# --- checagem rápida ---
print(df.shape)
print(df['label'].value_counts())
df['text_length'] = df['review_text'].str.len()
print("Tamanho médio do texto:", df['text_length'].mean())

In [ ]:
# --- gráfico 1: distribuição das classes (0 = negativo, 1 = positivo) ---
plt.figure(figsize=(6,4))
sns.countplot(x='label', data=df)
plt.title('Distribuição das classes')
plt.xlabel('label'); plt.ylabel('count')
plt.show()

In [ ]:
# --- gráfico 2: distribuição do comprimento das resenhas ---
plt.figure(figsize=(6,4))
sns.histplot(df['text_length'], bins=40, kde=True)
plt.title('Distribuição do tamanho das resenhas')
plt.xlabel('nº de caracteres')
plt.show()

In [ ]:
"""**Reasoning**:
The previous code block failed due to a `ValueError` when trying to compute the correlation matrix because of a string column ('submission_date').  I need to handle this error by excluding non-numeric columns from the correlation calculation.  I'll also improve the plot sizes and add labels and titles to the plots.


"""

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

In [ ]:
# ----- carregar e garantir text_length -----
df = pd.read_csv("archive/books_reviews.csv")
if "text_length" not in df.columns:
    df["text_length"] = df["review_text"].str.len()

In [ ]:
# ----- distribuição do rótulo (única categórica real) -----
plt.figure(figsize=(6,4))
sns.countplot(x="label", data=df)
plt.title("Distribuição das classes (0 = neg, 1 = pos)")
plt.xlabel("label"); plt.ylabel("count")
plt.show()

In [ ]:
# ----- histograma + boxplot do comprimento do texto -----
plt.figure(figsize=(6,4))
sns.histplot(df["text_length"], bins=40, kde=True)
plt.title("Histograma do tamanho das resenhas"); plt.xlabel("nº de caracteres")
plt.show()

In [ ]:
plt.figure(figsize=(6,4))
sns.boxplot(x=df["text_length"])
plt.title("Box-plot do tamanho das resenhas")
plt.show()

In [ ]:
# ----- (opcional) matriz de correlação se houver >1 numérico -----
num_cols = df.select_dtypes(include="number").columns
if len(num_cols) > 1:
    corr = df[num_cols].corr()
    plt.figure(figsize=(6,4))
    sns.heatmap(corr, annot=True, cmap="coolwarm", fmt=".2f")
    plt.title("Correlação entre variáveis numéricas")
    plt.show()
else:
    print("Só há uma coluna numérica; heatmap de correlação não é necessário.")

In [ ]:
"""# **Aqui começa a implementação da rede neural**"""

Implementação da rede neural para dados tabulares (Banco)

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, accuracy_score
import time

Carregar os dados pré-processados se ainda não estiverem na memória
Certifique-se de que X_train, X_test, y_train, y_test estão disponíveis do pré-processamento anterior

In [ ]:
# Arquitetura 1: MLP Básico com Scikit-learn
print("--- Treinando MLP Básico com Scikit-learn ---")
start_time = time.time()

In [ ]:
# Parâmetros: hidden_layer_sizes (número de neurônios por camada), max_iter (épocas)
mlp_sk = MLPClassifier(hidden_layer_sizes=(100,), max_iter=300, random_state=42)
mlp_sk.fit(X_train, y_train)

In [ ]:
y_pred_sk = mlp_sk.predict(X_test)

In [ ]:
print("Relatório de Classificação (MLP Scikit-learn):")
print(classification_report(y_test, y_pred_sk, zero_division=0))
print("Acurácia (MLP Scikit-learn):", accuracy_score(y_test, y_pred_sk))
print(f"Tempo de treinamento: {time.time() - start_time:.2f} segundos")
print("-" * 40)

Implementação da rede neural para dados de imagem (tf_flowers)

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt

Certifique-se de que ds_train, ds_val, ds_test estão disponíveis do carregamento anterior

In [ ]:
# Configurações para o pipeline de dados
IMG_HEIGHT = 180
IMG_WIDTH = 180
BATCH_SIZE = 32

In [ ]:
# Função para redimensionar e normalizar as imagens
def preprocess_image(image, label):
  image = tf.image.resize(image, (IMG_HEIGHT, IMG_WIDTH))
  image = image / 255.0  # Normalização
  return image, label

In [ ]:
ds_train_processed = ds_train.map(preprocess_image).batch(BATCH_SIZE).prefetch(buffer_size=tf.data.AUTOTUNE)
ds_val_processed = ds_val.map(preprocess_image).batch(BATCH_SIZE).prefetch(buffer_size=tf.data.AUTOTUNE)
ds_test_processed = ds_test.map(preprocess_image).batch(BATCH_SIZE).prefetch(buffer_size=tf.data.AUTOTUNE)

Visualizar algumas imagens (opcional)
for images, labels in ds_train_processed.take(1):
  plt.figure(figsize=(10, 10))
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy())
    plt.title(info.features['label'].names[labels[i].numpy()])
    plt.axis("off")
  plt.show()

In [ ]:
# Número de classes
num_classes = info.features['label'].num_classes
print(f"Número de classes para imagens: {num_classes}")

In [ ]:
# Arquitetura 2: CNN Simples
print("--- Treinando CNN Simples para Imagens ---")
start_time = time.time()

In [ ]:
model_cnn_simple = keras.Sequential([
    layers.Conv2D(32, 3, activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)),
    layers.MaxPooling2D(),
    layers.Conv2D(64, 3, activation='relu'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(num_classes, activation='softmax')
])

In [ ]:
model_cnn_simple.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

In [ ]:
history_cnn_simple = model_cnn_simple.fit(
    ds_train_processed,
    validation_data=ds_val_processed,
    epochs=10 # Pode ajustar o número de épocas
)

In [ ]:
print("Avaliando CNN Simples:")
loss_cnn_simple, accuracy_cnn_simple = model_cnn_simple.evaluate(ds_test_processed)
print(f"Loss no Teste: {loss_cnn_simple:.4f}")
print(f"Acurácia no Teste: {accuracy_cnn_simple:.4f}")
print(f"Tempo de treinamento: {time.time() - start_time:.2f} segundos")
print("-" * 40)

Implementação da rede neural para dados de texto (archive/books_reviews.csv)

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import numpy as np

Certifique-se de que o DataFrame df do archive/books_reviews.csv está disponível

In [ ]:
# Remover linhas com valores NaN na coluna 'review_text' ou 'label'
df.dropna(subset=['review_text', 'label'], inplace=True)

In [ ]:
# Separar textos e rótulos
texts = df['review_text'].tolist()
labels = df['label'].tolist()

In [ ]:
# Tokenização
vocab_size = 10000 # Tamanho do vocabulário
oov_tok = "<OOV>"  # Token para palavras fora do vocabulário
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

In [ ]:
# Padding
max_length = max([len(x) for x in sequences]) # Comprimento máximo das sequências
padded_sequences = pad_sequences(sequences, maxlen=max_length, padding='post', truncating='post')

In [ ]:
# Dividir em treino e teste
X_train_text, X_test_text, y_train_text, y_test_text = train_test_split(
    padded_sequences, labels, test_size=0.2, random_state=42, stratify=labels # estratifica para manter a proporção das classes
)

In [ ]:
y_train_text = np.array(y_train_text)
y_test_text = np.array(y_test_text)

In [ ]:
print(f"Tamanho do vocabulário: {vocab_size}")
print(f"Comprimento máximo das sequências: {max_length}")
print(f"Shape dos dados de treino (texto): {X_train_text.shape}")
print(f"Shape dos dados de teste (texto): {X_test_text.shape}")
print(f"Shape dos rótulos de treino (texto): {y_train_text.shape}")
print(f"Shape dos rótulos de teste (texto): {y_test_text.shape}")

In [ ]:
# Arquitetura 3: CNN Simples para Texto
print("--- Treinando CNN Simples para Texto ---")
start_time = time.time()

In [ ]:
embedding_dim = 16 # Dimensão do embedding

In [ ]:
model_cnn_text = keras.Sequential([
    layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    layers.Conv1D(128, 5, activation='relu'),
    layers.GlobalMaxPooling1D(),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid') # Sigmoid para classificação binária
])

In [ ]:
model_cnn_text.compile(optimizer='adam',
                       loss='binary_crossentropy',
                       metrics=['accuracy'])

In [ ]:
history_cnn_text = model_cnn_text.fit(
    X_train_text, y_train_text,
    epochs=10, # Pode ajustar o número de épocas
    validation_split=0.2 # Usar parte do treino para validação
)

In [ ]:
print("Avaliando CNN Simples para Texto:")
loss_cnn_text, accuracy_cnn_text = model_cnn_text.evaluate(X_test_text, y_test_text)
print(f"Loss no Teste: {loss_cnn_text:.4f}")
print(f"Acurácia no Teste: {accuracy_cnn_text:.4f}")
print(f"Tempo de treinamento: {time.time() - start_time:.2f} segundos")
print("-" * 40)

In [ ]:
# Arquitetura 4: MLP Mais Complexo com Keras (Dados Tabulares)
print("--- Treinando MLP Mais Complexo com Keras ---")
start_time = time.time()

Converter dados do scikit-learn para o formato do Keras se necessário
X_train_keras, X_test_keras, y_train_keras, y_test_keras = X_train, X_test, y_train, y_test # Se já estiverem no formato numpy/tensor

In [ ]:
model_mlp_keras = keras.Sequential([
    layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    layers.Dropout(0.3), # Dropout para regularização
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(32, activation='relu'),
    layers.Dense(1, activation='sigmoid') # Saída binária para o problema do banco
])

In [ ]:
model_mlp_keras.compile(optimizer='adam',
                        loss='binary_crossentropy',
                        metrics=['accuracy'])

O problema do banco é binário, então usamos binary_crossentropy

In [ ]:
# Usar um callback para parar o treino se a validação não melhorar
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [ ]:
history_mlp_keras = model_mlp_keras.fit(
    X_train, y_train,
    epochs=50, # Pode ajustar o número de épocas
    validation_split=0.2, # Usar parte do treino para validação
    callbacks=[early_stopping]
)

In [ ]:
print("Avaliando MLP Mais Complexo com Keras:")
loss_mlp_keras, accuracy_mlp_keras = model_mlp_keras.evaluate(X_test, y_test)
print(f"Loss no Teste: {loss_mlp_keras:.4f}")
print(f"Acurácia no Teste: {accuracy_mlp_keras:.4f}")
print(f"Tempo de treinamento: {time.time() - start_time:.2f} segundos")
print("-" * 40)

In [ ]:
# Arquitetura 5: CNN Mais Profunda para Imagens
print("--- Treinando CNN Mais Profunda para Imagens ---")
start_time = time.time()

In [ ]:
model_cnn_deep = keras.Sequential([
    layers.Conv2D(32, 3, activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)),
    layers.MaxPooling2D(),
    layers.Conv2D(64, 3, activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(128, 3, activation='relu'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.4),
    layers.Dense(num_classes, activation='softmax')
])

In [ ]:
model_cnn_deep.compile(optimizer='adam',
                       loss=tf.keras.losses.SparseCategoricalCrossentropy(),
                       metrics=['accuracy'])

In [ ]:
early_stopping_cnn = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [ ]:
history_cnn_deep = model_cnn_deep.fit(
    ds_train_processed,
    validation_data=ds_val_processed,
    epochs=20, # Pode ajustar o número de épocas
    callbacks=[early_stopping_cnn]
)

In [ ]:
print("Avaliando CNN Mais Profunda para Imagens:")
loss_cnn_deep, accuracy_cnn_deep = model_cnn_deep.evaluate(ds_test_processed)
print(f"Loss no Teste: {loss_cnn_deep:.4f}")
print(f"Acurácia no Teste: {accuracy_cnn_deep:.4f}")
print(f"Tempo de treinamento: {time.time() - start_time:.2f} segundos")
print("-" * 40)

In [ ]:
# Arquitetura 6: CNN + Camada Recorrente (LSTM) para Texto
print("--- Treinando CNN + LSTM para Texto ---")
start_time = time.time()

In [ ]:
embedding_dim = 32 # Pode ajustar a dimensão do embedding

In [ ]:
model_cnn_lstm_text = keras.Sequential([
    layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    layers.Conv1D(128, 5, activation='relu'),
    layers.MaxPooling1D(),
    layers.LSTM(64), # Camada LSTM
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid') # Saída binária
])

In [ ]:
model_cnn_lstm_text.compile(optimizer='adam',
                            loss='binary_crossentropy',
                            metrics=['accuracy'])

In [ ]:
early_stopping_text = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [ ]:
history_cnn_lstm_text = model_cnn_lstm_text.fit(
    X_train_text, y_train_text,
    epochs=10, # Pode ajustar o número de épocas
    validation_split=0.2,
    callbacks=[early_stopping_text]
)

In [ ]:
print("Avaliando CNN + LSTM para Texto:")
loss_cnn_lstm_text, accuracy_cnn_lstm_text = model_cnn_lstm_text.evaluate(X_test_text, y_test_text)
print(f"Loss no Teste: {loss_cnn_lstm_text:.4f}")
print(f"Acurácia no Teste: {accuracy_cnn_lstm_text:.4f}")
print(f"Tempo de treinamento: {time.time() - start_time:.2f} segundos")
print("-" * 40)

In [ ]:
# Arquitetura 7: Rede Recorrente (GRU) para Texto
print("--- Treinando Rede Recorrente (GRU) para Texto ---")
start_time = time.time()

In [ ]:
embedding_dim = 32 # Pode ajustar a dimensão do embedding

In [ ]:
model_gru_text = keras.Sequential([
    layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    layers.GRU(64), # Camada GRU
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid') # Saída binária
])

In [ ]:
model_gru_text.compile(optimizer='adam',
                       loss='binary_crossentropy',
                       metrics=['accuracy'])

In [ ]:
early_stopping_gru = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [ ]:
history_gru_text = model_gru_text.fit(
    X_train_text, y_train_text,
    epochs=10, # Pode ajustar o número de épocas
    validation_split=0.2,
    callbacks=[early_stopping_gru]
)

In [ ]:
print("Avaliando Rede Recorrente (GRU) para Texto:")
loss_gru_text, accuracy_gru_text = model_gru_text.evaluate(X_test_text, y_test_text)
print(f"Loss no Teste: {loss_gru_text:.4f}")
print(f"Acurácia no Teste: {accuracy_gru_text:.4f}")
print(f"Tempo de treinamento: {time.time() - start_time:.2f} segundos")
print("-" * 40)